In [1]:
from google.colab import files
uploaded = files.upload()

Saving MRL_dataset.zip to MRL_dataset.zip


In [4]:
file_name = list(uploaded.keys())[0]  # Get the filename from uploaded.keys()

from zipfile import ZipFile

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print("Done")

Done


In [5]:
!pip install tensorflow keras --upgrad
!pip3 install tensorflow keras --upgrad


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

ambiguous option: --upgrad (--upgrade, --upgrade-strategy?)

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

ambiguous option: --upgrad (--upgrade, --upgrade-strategy?)


In [6]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time
import os

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [9]:
train_generator = train_datagen.flow_from_directory(
    '/content/data',
    target_size=(24, 24),
    batch_size=32,
    color_mode='grayscale',
    class_mode='binary',
    subset='training'
)

Found 67919 images belonging to 3 classes.


In [10]:
validation_generator = train_datagen.flow_from_directory(
    '/content/data',
    target_size=(24, 24),
    batch_size=32,
    color_mode='grayscale',
    class_mode='binary',
    subset='validation'
)

Found 16979 images belonging to 3 classes.


In [11]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(24,24,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(train_generator, validation_data=validation_generator, epochs=10)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2123/2123 ━━━━━━━━━━━━━━━━━━━━ 98s 45ms/step - accuracy: 0.6013 - loss: 0.1173 - val_accuracy: 0.6000 - val_loss: -0.1837
Epoch 2/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 81s 38ms/step - accuracy: 0.5994 - loss: 0.3561 - val_accuracy: 0.6000 - val_loss: -0.0719
Epoch 3/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 81s 38ms/step - accuracy: 0.5996 - loss: 0.0838 - val_accuracy: 0.6000 - val_loss: -0.0391
Epoch 4/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 82s 38ms/step - accuracy: 0.6016 - loss: -0.0123 - val_accuracy: 0.6000 - val_loss: -0.0082
Epoch 5/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 81s 38ms/step - accuracy: 0.5993 - loss: 0.0713 - val_accuracy: 0.6000 - val_loss: -0.0198
Epoch 6/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 80s 38ms/step - accuracy: 0.5976 - loss: 0.1306 - val_accuracy: 0.6000 - val_loss: -2.5179e-04
Epoch 7/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 85s 40ms/step - accuracy: 0.5973 - loss: -0.0236 - val_accuracy: 0.6000 - val_loss: 2.4506e-04
Epoch 8/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 80s 38ms/step - accuracy: 0.6

In [14]:
model.save('drowsiness_model.h5')

In [15]:
model = tf.keras.models.load_model('drowsiness_model.h5')

In [16]:
def detect_drowsiness():
    cap = cv2.VideoCapture(0)
    closed_eyes_count = 0
    start_time = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray)

            for (ex, ey, ew, eh) in eyes:
                eye = roi_gray[ey:ey+eh, ex:ex+ew]
                eye = cv2.resize(eye, (24, 24)) / 255.0
                eye = np.reshape(eye, (1, 24, 24, 1))
                prediction = model.predict(eye)

                if prediction < 0.5:  # Assuming closed eyes are labeled as 0
                    if start_time is None:
                        start_time = time.time()
                    elif time.time() - start_time >= 0.25:
                        print("ALERT! Wake up!")
                        os.system('say "Wake up!"')  # MacOS; Use 'beep' or other audio command on Windows/Linux
                else:
                    start_time = None

        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Run detection
detect_drowsiness()